In [20]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance

Ideas for feature engineering

1.Look into ratios between key nutrients, maybe focus on those most correlated with the target nutrients

2.Make composite nutrient indicators, like adding all the macro nutrients as a total, or micro nutrient total

3.Split soil (knit) in to the constituent amounts of clay, sand and silt

4.Look for nutrients that are largely affected by amount of clay, silt and clay and create composite features

5.Use PCA and Kmeans with one hot encoded categorical variables and numerical data, then see if dropping the categorical columns helps with the model performance



Specific features to experiment with

N:S ratio

N+K

Maybe just do total macronutrients as a feature N+P+K

Also try weighted averages and geometric means of NPK

Ratios between the macro nutrients N/P N/K K/P

Kmeans clusters



In [22]:
ipage = pd.read_csv('./ipage_scaled.csv')
isda = pd.read_csv('./isda_scaled.csv')
isda = isda.dropna()
grevena = pd.read_csv('./grevena_scaled.csv')

In [23]:
# Split datasets
ipage_X = ipage.drop(columns=['SOC', 'Boron', 'Zinc'])
isda_X = isda.drop(columns=['SOC', 'Boron', 'Zinc'])
grevena_X = isda.drop(columns=['SOC', 'Boron', 'Zinc'])
ipage_y = ipage[['SOC', 'Boron', 'Zinc']]
isda_y = isda[['SOC', 'Boron', 'Zinc']]
grevena_y = isda[['SOC', 'Boron', 'Zinc']]

# Train/test split
ipage_X_train, ipage_X_test, ipage_y_train, ipage_y_test = train_test_split(ipage_X, ipage_y, test_size=0.2, random_state=0)
isda_X_train, isda_X_test, isda_y_train, isda_y_test = train_test_split(isda_X, isda_y, test_size=0.2, random_state=0)
grevena_X_train, grevena_X_test, grevena_y_train, grevena_y_test = train_test_split(grevena_X, grevena_y, test_size=0.2, random_state=0)

# Initialize dictionary to store results
results = {}

# Define function for evaluating models
def evaluate_model(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred, multioutput='raw_values')
    mae = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
    r2 = r2_score(y_true, y_pred, multioutput='variance_weighted')
    
    print(f"{model_name} Performance:")
    print(f"MSE: {mse}")
    print(f"MAE: {mae}")
    print(f"R² Score: {r2}")
    
    results[model_name] = {'MSE': mse, 'MAE': mae, 'R² Score': r2}
    print("-" * 30)

# Define models to use
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "XGBoost": XGBRegressor(objective='reg:squarederror')
}

# Loop through datasets
for dataset_name, (X_train, X_test, y_train, y_test) in {
    "ipage": (ipage_X_train, ipage_X_test, ipage_y_train, ipage_y_test),
    "isda": (isda_X_train, isda_X_test, isda_y_train, isda_y_test),
    "grevena": (grevena_X_train, grevena_X_test, grevena_y_train, grevena_y_test)
}.items():
    
    print(f"Evaluating models on {dataset_name} dataset:")
    
    # Loop through models
    for model_name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test)
        
        # Evaluate the model
        evaluate_model(y_test, y_pred, f"{model_name} on {dataset_name}")

        perm_importance = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42)
        importance_df = pd.DataFrame({
            'Feature': X_train.columns,
            'Permutation Importance': perm_importance.importances_mean
        }).sort_values(by='Permutation Importance', ascending=False)
        print(f"Permutation Importance for {model_name}:")
        print(importance_df)
        print("-" * 30)

Evaluating models on ipage dataset:
Linear Regression on ipage Performance:
MSE: [0.55115819 1.09809533 1.09273011]
MAE: [0.55087117 0.74940987 0.77552514]
R² Score: 0.20443674367107761
------------------------------
Permutation Importance for Linear Regression:
      Feature  Permutation Importance
0    Nitrogen                0.360336
1  Phosphorus                0.048618
2   Potassium                0.024098
4          pH                0.016393
3      Sulfur                0.012451
------------------------------
Random Forest on ipage Performance:
MSE: [0.49212619 1.055702   1.05695512]
MAE: [0.54362925 0.74820521 0.71404176]
R² Score: 0.24424425186763754
------------------------------
Permutation Importance for Random Forest:
      Feature  Permutation Importance
0    Nitrogen                0.486510
3      Sulfur                0.082087
1  Phosphorus                0.060164
2   Potassium                0.033433
4          pH                0.022459
------------------------------


It appears that SOC is easier to predict than Zn and B with just our numerical categories, the ipage dataset wasn't as easy to predict using just the numerical variables as the isda dataset. This is in line with the correlation matrices I saw during my EDA where the isda data showed much stronger correlations than the ipage. It reassuring to see that in both datasets Nitrogen was the most important feature, but I'd like to see whether some engineered features that capture more of the variance can be achieved.

Key takeaways:

- Nitrogen is the most important numerical variable
- Since SOC is important for the ability of soil to hold nutrients, maybe if we can achieve a low enough error in predicting SOC, we can use it in a subsequent model to predict Zn and B?
- I'd like to incorporate the categorical variables, but I'd like to get sand, silt and clay as numerical variables and combine them with some of the nurtient variables